In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import feature_functions
import tradestats
from feature_functions import *
import pandas as pd
from datetime import datetime

In [3]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
import graphviz

In [4]:
importlib.reload(feature_functions)
from feature_functions import *

In [5]:
prices = loaddata_4h('AUDUSD_Candlestick_4_Hour_BID_03.08.2003-04.09.2021_EEF_GMT.csv')

In [6]:
rsiKey = [13]
tdiKey = [13]
dictList = []
colFeat = []
keyList = []
# rsiDict   = rsi(prices, rsiKey)
tdiDict   = tdi1(prices, tdiKey)
# dictList.append(rsiDict.df); colFeat.append('rsi'); keyList.append(rsiKey) 
dictList.append(tdiDict.df); colFeat.append('tdi'); keyList.append(tdiKey) 

In [7]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','date','hour','weekday','open','high','low','close','volume']] = prices[['id','year','date','hour','weekday','open','high','low','close','volume']]   

In [8]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [9]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [10]:
masterFrame = masterFrame.dropna()

In [11]:
# masterFrame.iloc[0:-50,:20]

In [12]:
importlib.reload(tradestats)
from tradestats import *

In [13]:
starttime = datetime.now()
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.003],False)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5,9],[13,17],[0.001,0.002,0.003,0.004,0.005,0.006],True)
alltrades = preparetrades(masterFrame,[1],[5],[13,17],[0.003])
trades = cleartrades(alltrades.copy(),False)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.002],True)
endtime = datetime.now()
print(str(endtime - starttime))

0:00:02.461520


In [2112]:
trades.head(2)

,hour,weekday,tdi13rsi1,tdi13red1,tdi13green1,tdi13mid1,tdi13green1_red1,tdi13red2,tdi13green2,tdi13mid2,...,tdi13green_red_dist,tdi13mid_slope,tdi13habarsize1,tdi13hacolor1,tdi13barnumber1,tdi13habarsize2,tradetype,sl,closehour,profit1
88,5,5,33.858234,41.248787,33.858234,50.468543,-7.390554,43.762442,36.557539,51.313657,...,-7.297728,-0.845114,-12.2750,-1,2,-12.1250,1,20.0,13,-1
250,5,1,35.938428,40.940774,35.938428,55.977556,-5.002346,42.840413,39.754583,57.296712,...,-4.044088,-1.319156,-15.2125,-1,2,-23.6125,1,20.0,13,-1


In [2035]:
X = trades.iloc[:, :-1].values
feature_names = trades.columns[:-1]
y = trades.iloc[:, -1:].values

In [2036]:
# regr = DecisionTreeRegressor(min_samples_leaf=0.001,criterion='mse',max_depth=17, random_state=1234)
# criterion{“mse”, “friedman_mse”, “mae”, “poisson”}, default=”mse”
regr = DecisionTreeClassifier(min_samples_leaf=1,criterion='gini',max_depth=8, random_state=1234)
# criterion{“gini”, “entropy”}, default=”gini”
regr.fit(X, y)
score = regr.score(X, y)
print(score)

0.7924017332095327


In [2037]:
"""
all features mse depth 8 leaf 0.05 =0,0049
wyciąłem poniższe mse depth 8 leaf 0.05 =0,0043
#     df['green1_red1'] = df.green1 - df.red1
#     df['green2_red2'] = df.green2 - df.red2
#     df['green_red_change'] = df.green1_red1 - df.green2_red2
#     df['green_red_slope_change'] = df.green_slope - df.red_slope
all features mse depth 8 leaf 0.1 =0.0028
all features friedman_mse depth 8 leaf 0.05 =0.0049
all features mse depth 8 leaf 0.03 = 0.0069
all features mse depth 8 leaf 0.01 = 0.0074
all features mse depth 8 leaf 1 = 0.048
all features mse depth 9 leaf 1 = 0.051
all features mse depth 15 leaf 1 = 0.11
all features mse depth 15 leaf 0.001 = 0.054
all features mse depth 15 leaf 0.05 = 0,028
all features mse depth 5 leaf 0.05 = 0.0016
"""

"\nall features mse depth 8 leaf 0.05 =0,0049\nwyciąłem poniższe mse depth 8 leaf 0.05 =0,0043\n#     df['green1_red1'] = df.green1 - df.red1\n#     df['green2_red2'] = df.green2 - df.red2\n#     df['green_red_change'] = df.green1_red1 - df.green2_red2\n#     df['green_red_slope_change'] = df.green_slope - df.red_slope\nall features mse depth 8 leaf 0.1 =0.0028\nall features friedman_mse depth 8 leaf 0.05 =0.0049\nall features mse depth 8 leaf 0.03 = 0.0069\nall features mse depth 8 leaf 0.01 = 0.0074\nall features mse depth 8 leaf 1 = 0.048\nall features mse depth 9 leaf 1 = 0.051\nall features mse depth 15 leaf 1 = 0.11\nall features mse depth 15 leaf 0.001 = 0.054\nall features mse depth 15 leaf 0.05 = 0,028\nall features mse depth 5 leaf 0.05 = 0.0016\n"

In [2038]:
dd = pd.DataFrame()
dd['f'] = feature_names.values
dd['i'] = regr.feature_importances_
dd.sort_values(by=['i'], ascending=False)

,f,i
24,sl,0.133956
22,tdi13habarsize2,0.111599
9,tdi13mid2,0.098214
17,tdi13green_red_dist,0.091681
19,tdi13habarsize1,0.085841
14,tdi13red_slope,0.076966
7,tdi13red2,0.068794
3,tdi13red1,0.056730
25,closehour,0.052563
10,tdi13green2_red2,0.048402


In [2039]:
# text_representation = tree.export_text(regr)
# print(text_representation)

In [2040]:
from graphviz import Source
from sklearn import tree
graph = Source( tree.export_graphviz(regr, out_file=None, feature_names=feature_names,rounded = True, proportion = False, 
                 filled = True))
graph.format = 'png'
graph.render('dtree_render',view=True)

'dtree_render.png'

In [2062]:
alltrades.head()

,id,year,date,hour,weekday,open,high,low,close,volume,...,openprice,openindex,sl,closeindex,closeprice,closehour,slindex,slprice,profit,profit1
32,32,2003,2003-08-11 01:00:00,1,1,0.65080,0.65370,0.65014,0.65305,19529.3003,...,NaN,NaN,10.0,-1.0,-1.00000,-1,-1.0,-1.0,0.0,-1
33,33,2003,2003-08-11 05:00:00,5,1,0.65282,0.65495,0.65222,0.65348,26580.2300,...,0.65282,33.0,10.0,35.0,0.65645,13,-1.0,-1.0,36.3,1
34,34,2003,2003-08-11 09:00:00,9,1,0.65353,0.65715,0.65249,0.65650,44046.8593,...,NaN,NaN,10.0,-1.0,-1.00000,-1,-1.0,-1.0,0.0,-1
35,35,2003,2003-08-11 13:00:00,13,1,0.65645,0.65952,0.65548,0.65886,20403.6897,...,NaN,NaN,10.0,-1.0,-1.00000,-1,-1.0,-1.0,0.0,-1
36,36,2003,2003-08-11 17:00:00,17,1,0.65922,0.65948,0.65731,0.65782,10956.1697,...,NaN,NaN,10.0,-1.0,-1.00000,-1,-1.0,-1.0,0.0,-1


In [2146]:
importlib.reload(tradestats)
from tradestats import *
stats = stattrades(alltrades)

In [2147]:
stats.sort_values("profit_ratio",ascending=False)

,tradetype,hour,closehour,sl,counts,profit_mean,profit_sum,profit_ratio,maxdown,maxdown_ratio
1,1,5,13,20.0,4715,0.222312,1048.2,52.410000,-500.0,-25.000000
4,1,5,13,50.0,4715,0.431410,2034.1,40.682000,-610.3,-12.206000
5,1,5,13,60.0,4715,0.498473,2350.3,39.171667,-614.5,-10.241667
3,1,5,13,40.0,4715,0.332238,1566.5,39.162500,-490.3,-12.257500
9,1,5,17,40.0,4715,0.328441,1548.6,38.715000,-600.0,-15.000000
10,1,5,17,50.0,4715,0.353128,1665.0,33.300000,-740.5,-14.810000
7,1,5,17,20.0,4715,0.134507,634.2,31.710000,-565.3,-28.265000
2,1,5,13,30.0,4715,0.199661,941.4,31.380000,-370.3,-12.343333
8,1,5,17,30.0,4715,0.185281,873.6,29.120000,-720.9,-24.030000
11,1,5,17,60.0,4715,0.369162,1740.6,29.010000,-859.0,-14.316667


In [ ]:
importlib.reload(tradestats)
from tradestats import *

tradetypes = [1]
openhours = [5]
closehours = [13,17]
sls = [30] #,20,30,40,50,60
bar2froms = [-1000,-20,-10,0,10,20,1000]
bar2tos = [-1000,-20,-10,0,10,20,1000]
bar1froms = [-1000,-20,-10,0,10,20,1000]
bar1tos =[-1000,-20,-10,0,10,20,1000]
gr2froms = [-1000,-8,-3,0,3,8,1000]
gr2tos = [-1000,-8,-3,0,3,8,1000]
gr1froms = [-1000,-8,-3,0,3,8,1000]
gr1tos = [-1000,-8,-3,0,3,8,1000]
starttime = datetime.now()
stats,stats0 = stathyper(alltrades,tradetypes,openhours,closehours,sls,bar2froms,bar2tos,bar1froms,bar1tos,gr2froms,gr2tos,gr1froms,gr1tos)
endtime = datetime.now()
print(str(endtime - starttime))

tradetype 1
  openhour 5
    closehour 13
      sl 30
        bar2from -1000
          bar2to -1000
          bar2to -20
            bar1from -1000
              bar1to -1000
              bar1to -20
                gr2from -1000
                  gr2to -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
              bar1to -10
                g

                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
            bar1from -10
              bar1to -1000
              bar1to -20
              bar1to -10
              bar1to 0
                gr2from -1000
                  gr2to -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
         

                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
            bar1from 1000
              bar1to -1000
              bar1to -20
              bar1to -10
              bar1to 0
              bar1to 10
              bar1to 20
              bar1to 1000
          bar2to -10
            bar1from -1000
              bar1to -1000
              bar1to -20

                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
              bar1to 1000
                gr2from -1000
                  gr2to -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
   

                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
            bar1from 20
              bar1to -1000
              bar1to -20
              bar1to -10
              bar1to 0
              bar1to 10
              bar1to 20
              bar1to 1000
                gr2from -1000
                  gr2to -8
                  gr2to -3
                  gr2to 0
                  

...

                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
              bar1to 1000
                gr2from -1000
                  gr2to -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
    

                gr2from 1000
              bar1to 1000
                gr2from -1000
                  gr2to -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
            bar1from -20
              bar1to -1000
              bar1to -20
              bar1to -10
                gr2from -1000
                  gr2to -8
                  gr2to -3
 

                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
              bar1to 20
                gr2from -1000
                  gr2to -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
    

                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
              bar1to 20
                gr2from -1000
                  gr2to -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
              bar1to 1000
                gr2from -1000
                  gr2to -8
                  gr2t

                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from -3
                  gr2to 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 0
                  gr2to 3
                  gr2to 8
                  gr2to 1000
                gr2from 3
                  gr2to 8
                  gr2to 1000
                gr2from 8
                  gr2to 1000
                gr2from 1000
            bar1from 0
              bar1to -1000
              bar1to -20
              bar1to -10
              bar1to 0
              bar1to 10
                gr2from -1000
                  gr2to -8
                  gr2to -3
                  gr2to 0
                  gr2to 3
                

In [15]:
stats.sort_values("profit_sum",ascending=False)

,tradetype,openhour,closehour,sl,bar2from,bar2to,bar1from,bar1to,gr2from,gr2to,gr1from,gr1to,count,mean,profit_sum,maxdown,profit_ratio,maxdown_ratio
0,1,5,13,30,-1000,1000,-1000,1000,-1000,8,-3,8,3164,1.176580,3722.7,-420.0,124.09,-14
0,1,5,13,30,-1000,1000,-1000,1000,-8,8,-3,8,3163,1.160986,3672.2,-420.0,122.407,-14
0,1,5,17,30,-1000,1000,-1000,1000,-1000,3,-3,1000,2247,1.593769,3581.2,-450.0,119.373,-15
0,1,5,13,30,-1000,1000,-1000,1000,-1000,8,-3,1000,3267,1.093664,3573.0,-420.0,119.1,-14
0,1,5,13,30,-1000,1000,-1000,1000,-1000,1000,-3,8,3266,1.088702,3555.7,-480.0,118.523,-16
0,1,5,17,30,-1000,1000,-1000,1000,-8,3,-3,1000,2246,1.577605,3543.3,-450.0,118.11,-15
0,1,5,17,30,-1000,1000,-1000,1000,-1000,3,-3,8,2241,1.580678,3542.3,-450.0,118.077,-15
0,1,5,13,30,-1000,1000,-1000,1000,-8,8,-3,1000,3266,1.078536,3522.5,-420.0,117.417,-14
0,1,5,13,30,-1000,1000,-1000,1000,-8,1000,-3,8,3265,1.073568,3505.2,-480.0,116.84,-16
0,1,5,17,30,-1000,1000,-1000,1000,-8,3,-3,8,2240,1.564464,3504.4,-450.0,116.813,-15


In [ ]:
stats.to_csv(sep=';',path_or_buf='../Data/stats30.csv',date_format="%Y-%m-%d",index = False,na_rep='')